In [4]:
import io, os
from datetime import datetime, timezone, timedelta
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
from IPython.display import FileLink, display
import qrcode

def _fit_font(c, text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    size = max_font
    while size > min_font and pdfmetrics.stringWidth(t, font_name, size) > box_w:
        size -= 0.5
    return max(min_font, size)

def _qr_reader(value, box_size=6, border=2):
    q = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=box_size, border=border)
    q.add_data(value); q.make(fit=True)
    return ImageReader(q.make_image(fill_color="black", back_color="white").convert("RGB"))

def _num(x):
    try: return float(str(x))
    except: return None

def _neat_num(v):
    if v in [None, ""]: return ""
    try: return f"{float(str(v)):.2f}"
    except: return str(v)

def _humanize(key):
    s = (key or "").replace("_"," ").strip()
    return (s[:1].upper()+s[1:]) if s else ""

def _right_text(c, text, font, size, x_right, y, pad=0):
    t = "" if text is None else str(text)
    w = pdfmetrics.stringWidth(t, font, size)
    c.setFont(font, size)
    c.drawString(x_right - pad - w, y, t)

def _two_pumps_sorted(dispenser):
    order = ["regular","diesel"]
    by_name = {str(p.get("name","")).lower(): p for p in (dispenser.get("pumps") or [])}
    rows = []
    for nm in order:
        p = by_name.get(nm)
        if p is None:
            rows.append({"pump_name": nm, "inv": {}})
        else:
            rows.append({"pump_name": str(p.get("name","")), "inv": p.get("latest_inventory",{}) or {}})
    return rows

def _parse_iso_dt(s):
    if not s: return None
    try:
        if s.endswith("Z"):
            return datetime.fromisoformat(s.replace("Z","+00:00"))
        return datetime.fromisoformat(s)
    except:
        return None

def _ph_fmt(dt):
    if not dt: return "", ""
    try:
        ph = dt.astimezone(timezone(timedelta(hours=8)))
        return ph.strftime("%Y-%m-%d"), ph.strftime("%A")
    except:
        return dt.strftime("%Y-%m-%d"), dt.strftime("%A")

def render_pump_inventory_prefilled(prefill, dispensers, filename="loboc_pump_inventory_form_a4.pdf"):
    title = "Pump Inventory Record"
    barcode_key = "current_form_qr_code"
    page_size = A4
    w, h = page_size
    m = 10*mm
    pad = 2*mm
    left, right = m, w - m
    usable_w = right - left
    c = canvas.Canvas(filename, pagesize=page_size)
    c.setTitle(title)

    def draw_title():
        fs = _fit_font(c, title, "Helvetica-Bold", 16, 9, w - 2*m)
        c.setFont("Helvetica-Bold", fs)
        top_y = h - m - 4*mm
        c.drawCentredString(w/2, top_y, title)
        return top_y, fs

    def draw_qr_inline(cursor_y, title_fs):
        if not prefill.get(barcode_key): return cursor_y - 4
        qr = _qr_reader(str(prefill[barcode_key]), box_size=5, border=2)
        box = 14*mm
        y_top = cursor_y - title_fs - 2*mm
        x = left
        c.drawImage(qr, x, y_top - box, width=box, height=box, preserveAspectRatio=True, anchor='sw', mask='auto')
        c.setFont("Helvetica-Bold", 9)
        c.drawString(x + box + 3, y_top - 9, f"QR: {str(prefill[barcode_key])}")
        return y_top - box - 3

    def draw_row(cursor_y, cols, row_h):
        c.setLineWidth(1.0)
        c.rect(left, cursor_y - row_h, usable_w, row_h, stroke=1, fill=0)
        x = left
        acc = 0.0
        for i, (field, ratio) in enumerate(cols):
            acc += ratio
            if i < len(cols) - 1:
                x_next = left + usable_w * acc
                c.line(x_next, cursor_y - row_h, x_next, cursor_y)
            col_w = usable_w * ratio
            key = field.get("key")
            label = field.get("label") or _humanize(key)
            fs = _fit_font(c, label, "Helvetica", 9, 7, col_w - 2*pad)
            c.setFont("Helvetica", fs)
            c.drawString(x + pad, cursor_y - pad - fs*0.2, label)
            if key in prefill and str(prefill.get(key) or "") != "":
                val = str(prefill[key])
                vfs = _fit_font(c, val, "Helvetica-Bold", 11, 7.5, col_w - 2*pad)
                c.setFont("Helvetica-Bold", vfs)
                vy = cursor_y - (row_h/2) - (vfs*0.35)
                c.drawString(x + pad, vy, val)
            x = left + usable_w * acc
        return cursor_y - row_h

    def draw_group_title(cursor_y, text):
        fs = _fit_font(c, text, "Helvetica-Bold", 11, 8.5, usable_w)
        c.setFont("Helvetica-Bold", fs)
        c.drawString(left, cursor_y - fs - 1, text)
        return cursor_y - (fs + 5)



    def draw_pump_card_at(x, cursor_y, card_w, dispenser_name, pump_name, inv):
        title_h = 7.5*mm
        row_h = 6.3*mm
        gap = 2*mm
        price = _num(inv.get("price"))
        cash = _num(inv.get("cash"))
        po = _num(inv.get("po"))
        beginning = _num(inv.get("beginning_inventory"))   # ← NEW
        ending = _num(inv.get("ending_inventory"))
        s = _num(inv.get("starting_liter_meter"))
        e = _num(inv.get("ending_liter_meter"))
        sales_l = (e - s) if (s is not None and e is not None) else None
        sales_p = (sales_l * price) if (sales_l is not None and price is not None) else None

        # Beginning Inventory inserted right after Product
        lines = [
            ("Product", inv.get("product","")),
            ("Beginning Inventory (L)", _neat_num(beginning)),   # ← NEW ROW
            ("Price (Php/L)", _neat_num(price)),
            ("Cash (Php)", _neat_num(cash)),
            ("PO (Php)", _neat_num(po)),
            ("Ending Inventory (L)", _neat_num(ending)),
        ]

        meter_lines = [
            ("Starting Liter Meter", _neat_num(s)),
            ("Ending Liter Meter", _neat_num(e)),
            ("Sales (L)", _neat_num(sales_l) if sales_l is not None else ""),
            ("Sales (Php)", _neat_num(inv.get("sales","")) if inv.get("sales") not in [None,""] else (_neat_num(sales_p) if sales_p is not None else "")),
        ]

        total_h = title_h + gap + (len(lines)+len(meter_lines)) * row_h + gap
        c.setLineWidth(1.0)
        c.roundRect(x, cursor_y - total_h, card_w, total_h, 5, stroke=1, fill=0)
        c.setFont("Helvetica-Bold", 10)
        c.drawString(x + pad, cursor_y - title_h + 1.5, f"{dispenser_name.upper()} — {pump_name.upper()}")

        y = cursor_y - title_h - gap
        lab_w = card_w * 0.52
        val_w = card_w * 0.48
        for (lab, val) in lines:
            c.setFont("Helvetica", 8.7)
            c.drawString(x + pad, y - 8/2, lab)
            fs = _fit_font(c, str(val), "Helvetica-Bold", 10.5, 7.5, val_w - 2*pad)
            _right_text(c, str(val), "Helvetica-Bold", fs, x + lab_w + val_w, y - 8/2, pad)
            y -= row_h

        c.setStrokeColor(colors.gray)
        c.line(x + pad, y, x + card_w - pad, y)
        c.setStrokeColor(colors.black)
        y -= gap

        for (lab, val) in meter_lines:
            c.setFont("Helvetica", 8.7)
            c.drawString(x + pad, y - 8/2, lab)
            fs = _fit_font(c, str(val), "Helvetica-Bold", 10.5, 7.5, val_w - 2*pad)
            _right_text(c, str(val), "Helvetica-Bold", fs, x + lab_w + val_w, y - 8/2, pad)
            y -= row_h

        return total_h





    ty, tfs = draw_title()
    cursor = draw_qr_inline(ty, tfs)

    header1 = [({"key":"form_name","label":"Form Name"},0.35),({"key":"generated_on","label":"Generated On"},0.25),({"key":"generated_by","label":"Generated By"},0.20),({"key":"date","label":"Date"},0.20)]
    header2 = [({"key":"date_name","label":"Date (humanized)"},0.30),({"key":"current_form_qr_code","label":"Current Form QR Code"},0.30),({"key":"location","label":"Location"},0.20),({"key":"cashier_employee_number","label":"Cashier Employee No."},0.20)]
    cursor = draw_row(cursor, header1, row_h=12.0*mm)
    cursor = draw_row(cursor, header2, row_h=12.0*mm)

    try:
        disps = sorted(list(dispensers), key=lambda d: int(d.get("id")))
    except:
        disps = sorted(list(dispensers), key=lambda d: str(d.get("id")))
    groups = [("FUEL DISPENSER 1 OF 2", disps[0] if len(disps)>0 else None),
              ("FUEL DISPENSER 2 OF 2", disps[1] if len(disps)>1 else None)]

    gap_col = 6*mm
    cols = 2
    card_w = (usable_w - gap_col) / cols

    for group_title, disp in groups:
        if not disp: continue
        cursor = draw_group_title(cursor, group_title + f" — {disp.get('name','').upper()}")
        pumps = _two_pumps_sorted(disp)
        x1 = left
        x2 = left + card_w + gap_col
        h1 = draw_pump_card_at(x1, cursor, card_w, disp.get("name",""), pumps[0]["pump_name"], pumps[0]["inv"])
        h2 = draw_pump_card_at(x2, cursor, card_w, disp.get("name",""), pumps[1]["pump_name"], pumps[1]["inv"])
        cursor = cursor - max(h1, h2) - 6

    sign_fields = [({"key":"verified_by","label":"Verified By"},0.33),({"key":"cashier_employee_number","label":"Cashier Employee No."},0.33),({"key":"recorder_employee_number","label":"Recorder Employee No."},0.34)]
    cursor = draw_row(cursor, sign_fields, row_h=18.0*mm)

    c.save()
    try: display(FileLink(filename))
    except: pass
    return os.path.abspath(filename)

loboc_api_response = [
  {
    "id": "4",
    "name": "dispenser-four",
    "location": "loboc",
    "pumps": [
      {
        "id": 7,
        "name": "regular",
        "latest_inventory": {
          "id": "0e726124-abf7-4ebc-9206-77344f0c34f2",
          "created_at": "2025-09-27T19:26:35.056000Z",
          "location": "loboc",
          "date": "2025-09-27T16:00:00Z",
          "dispenser_name": "dispenser-four",
          "pump_id": "7",
          "pump_name": "regular",
          "product": "regular",
          "unit": "liter",
          "price": "62.50",
          "beginning_inventory": "5000.00",
          "calibration": "0.00",
          "po": "0.00",
          "cash": "0.00",
          "ending_inventory": "5000.00",
          "starting_liter_meter": "26309.00",
          "ending_liter_meter": "26309.00"
        }
      },
      {
        "id": 8,
        "name": "diesel",
        "latest_inventory": {
          "id": "d8f37a79-c974-45be-b7ad-0736177f1d11",
          "created_at": "2025-09-27T19:26:35.056000Z",
          "location": "loboc",
          "date": "2025-09-27T16:00:00Z",
          "dispenser_name": "dispenser-four",
          "pump_id": "8",
          "pump_name": "diesel",
          "product": "diesel",
          "unit": "liter",
          "price": "58.00",
          "beginning_inventory": "5000.00",
          "calibration": "0.00",
          "po": "0.00",
          "cash": "0.00",
          "ending_inventory": "5000.00",
          "starting_liter_meter": "45398.00",
          "ending_liter_meter": "45398.00"
        }
      }
    ]
  },
  {
    "id": "3",
    "name": "dispenser-three",
    "location": "loboc",
    "pumps": [
      {
        "id": 5,
        "name": "regular",
        "latest_inventory": {
          "id": "9ffc1d76-c0b8-412c-ad40-7ccccc4d6660",
          "created_at": "2025-09-27T19:26:35.056000Z",
          "location": "loboc",
          "date": "2025-09-27T16:00:00Z",
          "dispenser_name": "dispenser-three",
          "pump_id": "5",
          "pump_name": "regular",
          "product": "regular",
          "unit": "liter",
          "price": "62.50",
          "beginning_inventory": "5000.00",
          "calibration": "0.00",
          "po": "0.00",
          "cash": "0.00",
          "ending_inventory": "5000.00",
          "starting_liter_meter": "30039.00",
          "ending_liter_meter": "30039.00"
        }
      },
      {
        "id": 6,
        "name": "diesel",
        "latest_inventory": {
          "id": "4cd19ec0-ed64-4b4b-ba20-ab9929de0214",
          "created_at": "2025-09-27T19:26:35.056000Z",
          "location": "loboc",
          "date": "2025-09-27T16:00:00Z",
          "dispenser_name": "dispenser-three",
          "pump_id": "6",
          "pump_name": "diesel",
          "product": "diesel",
          "unit": "liter",
          "price": "58.00",
          "beginning_inventory": "5000.00",
          "calibration": "0.00",
          "po": "0.00",
          "cash": "0.00",
          "ending_inventory": "5000.00",
          "starting_liter_meter": "47710.00",
          "ending_liter_meter": "47710.00"
        }
      }
    ]
  }
]

def _infer_date_and_location(api_items):
    dates = []
    loc = None
    for d in api_items:
        for p in d.get("pumps", []):
            inv = p.get("latest_inventory") or {}
            dt = _parse_iso_dt(inv.get("date") or inv.get("created_at"))
            if dt: dates.append(dt)
            if not loc: loc = inv.get("location") or d.get("location")
    if dates:
        latest = max(dates)
        ds, day = _ph_fmt(latest)
    else:
        now = datetime.now(timezone(timedelta(hours=8)))
        ds, day = now.strftime("%Y-%m-%d"), now.strftime("%A")
    return ds, day, loc or ""

date_str, day_name, location = _infer_date_and_location(loboc_api_response)

prefill = {
    "form_name": "Pump Inventory Record",
    "generated_on": datetime.now(timezone(timedelta(hours=8))).strftime("%Y-%m-%d %H:%M"),
    "generated_by": "System",
    "date": date_str,
    "date_name": day_name,
    "current_form_qr_code": "86543219",
    "location": location.title(),
    "prepared_by": "",
    "verified_by": "",
    "approved_by": "",
    "remarks": ""
}

render_pump_inventory_prefilled(prefill, loboc_api_response, filename="loboc_pump_inventory_form_a4.pdf")


h:\github4\jefstore-gasstations-backend\reportlab\daily-pump-inventory-form\loboc_pump_inventory_form_a4.pdf

'h:\\github4\\jefstore-gasstations-backend\\reportlab\\daily-pump-inventory-form\\loboc_pump_inventory_form_a4.pdf'